# D1-D4 Single

In [1]:
import tensorflow as tf
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from keras.callbacks import EarlyStopping
from keras.layers import (Activation, Attention, Bidirectional, Concatenate, Conv1D,
                           Dense, Dropout, Embedding, Flatten, GlobalMaxPooling1D,
                           Input, Layer, LSTM, MaxPooling1D, Multiply, Permute,
                           RepeatVector, Reshape, SpatialDropout1D, TimeDistributed)
from keras.models import Model
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from sklearn.ensemble import ExtraTreesClassifier
import sys
sys.path.append('./cs16')
import cs16.prep as prep16
import cs16.plot as plot16
import cs16.build as build16
imagesize = 64

import pandas as pd

#data_type = input("Enter '1. single' for single dataset or '2. multi' for multiple dataset: ")
data_type = '1'
if data_type == '1':
    file_path = 'single.txt'
    folder_path = './data/MVSA/single/'
elif data_type == '2':
    file_path = 'multi.txt'
    folder_path = './data/MVSA/multiple/'
else:
    print("Invalid input. Please enter either 'single' or 'multi'.")
    exit()

df = pd.read_csv(file_path, index_col=None, encoding='ISO-8859-1')

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import pandas as pd
import matplotlib.pyplot as plt
# Download stopwords and punkt tokenizer if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Define a function to preprocess text
def nlp_text(text):
    # Convert text to lowercase
    text = text.lower()
    
    # Remove URLs using regex
    text = re.sub(r'http\S+', '', text)
    
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenize text into individual words
    words = word_tokenize(text)
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Stem words using Porter Stemmer
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]
    
    # Join words back into a single string
    text = ' '.join(words)
    
    return text

# Apply the preprocess_text function to the 'tweet' column of the dataframe
df['tweet'] = df['tweet'].apply(nlp_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ausco\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
X_text, y_text = prep16.preprocess_text(df)
X_train_text, X_val_text, X_test_text, \
y_train_text, y_val_text, y_test_text = prep16.split_data(X_text, y_text, random_state=42)

X_polar, y_polar = prep16.preprocess_text(df,label = 'polarity')
X_train_polar, X_val_polar, X_test_polar, \
y_train_polar, y_val_polar, y_test_polar = prep16.split_data(X_polar, y_polar, random_state=42)

image_data_s, image_label_s = prep16.preprocess_images(df, folder_path, imagesize)
y_s = to_categorical(image_label_s, num_classes=3)

X_train_image, X_val_image, X_test_image, \
y_train_image, y_val_image, y_test_image= prep16.split_data(image_data_s, y_s, random_state=42)

y_train = to_categorical(y_train_polar, num_classes=3)
y_val =to_categorical(y_val_polar, num_classes=3)
y_test =to_categorical(y_test_polar, num_classes=3)
file_path

'single.txt'

# D1- Late Fusion* 08V

In [4]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Softmax
from tensorflow.keras.models import Model
import numpy as np

# ----- Text branch -----
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=100)(text_input)
t = LSTM(64, activation='tanh')(t)
text_branch = Dense(64, activation='tanh')(t)
text_out = Dense(3, activation='softmax', name='text_softmax')(text_branch)

# ----- Image branch -----
image_input = Input(shape=(64, 64, 3), name='image_input')
i = Conv2D(64, (3, 3), activation='relu', padding='same')(image_input)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = Conv2D(128, (3, 3), activation='relu', padding='same')(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)
i = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
i = Dropout(0.5)(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)
i = Flatten()(i)
image_branch = Dense(64, activation='relu')(i)
image_out = Dense(3, activation='softmax', name='image_softmax')(image_branch)

# ----- Define Output model-----
model = Model(inputs=[text_input, image_input], outputs=[text_out, image_out])

# compile
model.compile(
    loss='categorical_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy']
)

# ---- Training
history = model.fit(
    [X_train_text, X_train_image], 
    [y_train, y_train],       # 如果兩分支label一樣就傳一份，否則可分開
    validation_data=([X_val_text, X_val_image], [y_val, y_val]),
    epochs=10,
    batch_size=64
)

# ----- Prediction & Late Fusion -----
# seperate Probs
text_probs, image_probs = model.predict([X_test_text, X_test_image])
# Late Fusion：AVG
ensemble_probs = (text_probs + image_probs) / 2.0
predictions = np.argmax(ensemble_probs, axis=1)
true_labels = np.argmax(y_test, axis=1)
print(classification_report(true_labels, predictions, digits=4))

Epoch 1/10
61/61 [==============================] - 26s 382ms/step - loss: 2.1501 - text_softmax_loss: 0.9829 - image_softmax_loss: 1.1672 - text_softmax_accuracy: 0.5435 - image_softmax_accuracy: 0.4816 - val_loss: 1.9655 - val_text_softmax_loss: 0.9498 - val_image_softmax_loss: 1.0158 - val_text_softmax_accuracy: 0.5483 - val_image_softmax_accuracy: 0.5462
Epoch 2/10
61/61 [==============================] - 23s 383ms/step - loss: 1.7258 - text_softmax_loss: 0.7127 - image_softmax_loss: 1.0131 - text_softmax_accuracy: 0.6860 - image_softmax_accuracy: 0.5284 - val_loss: 1.9816 - val_text_softmax_loss: 0.9423 - val_image_softmax_loss: 1.0392 - val_text_softmax_accuracy: 0.5811 - val_image_softmax_accuracy: 0.5483
Epoch 3/10
61/61 [==============================] - 24s 399ms/step - loss: 1.3530 - text_softmax_loss: 0.3959 - image_softmax_loss: 0.9572 - text_softmax_accuracy: 0.8490 - image_softmax_accuracy: 0.5612 - val_loss: 2.2085 - val_text_softmax_loss: 1.1460 - val_image_softmax_los

# D2 Early Fusion
(13/05/2025 test ok)

In [23]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate, Dropout, BatchNormalization, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np



start_time = time.time()

# Text branch
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=100)(text_input)
t = LSTM(64, activation='tanh')(t)  # (batch, 64)

# Image branch
image_input = Input(shape=(64, 64, 3), name='image_input')
i = Conv2D(64, (3, 3), activation='relu', padding='same')(image_input)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = Conv2D(128, (3, 3), activation='relu', padding='same')(i)
i = MaxPooling2D(pool_size=(2, 2))(i)
i = BatchNormalization()(i)

# Layer2:
#i = Conv2D(32, (3, 3), activation=image_af, padding='same')(i)
#i = Dropout(0.5)(i)  # Add a dropout layer
#i = MaxPooling2D(pool_size=(2, 2))(i)
#i = BatchNormalization()(i)

i = Flatten()(i)  # (batch, N)


# Early fusion: 
fusion = Concatenate()([t, i])  # (batch, 64+N)

# Further processing
fusion = Dense(128, activation='relu')(fusion)
fusion = Dropout(0.5)(fusion)
fusion = Dense(64, activation='relu')(fusion)
output = Dense(3, activation='softmax')(fusion)

model = Model(inputs=[text_input, image_input], outputs=output)
optimizer = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit([X_train_text, X_train_image], y_train, 
                    validation_data=([X_val_text, X_val_image], y_val),
                    epochs=10, 
                    batch_size=64)

# Evaluate the model
#test_loss, test_acc = model.evaluate([X_test_text, X_test_image], y_test)

from sklearn.metrics import classification_report
import numpy as np

predictions_prob = model.predict([X_test_text, X_test_image])


predictions = np.argmax(predictions_prob, axis=1)

true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))

#------------------------------------------------------------
output = some_function()
print(f"函數輸出: {output}")
end_time = time.time()

elapsed_time = end_time - start_time

print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")

Epoch 1/10
61/61 [==============================] - 32s 496ms/step - loss: 1.1153 - accuracy: 0.5181 - val_loss: 1.0673 - val_accuracy: 0.5462
Epoch 2/10
61/61 [==============================] - 37s 614ms/step - loss: 0.9887 - accuracy: 0.5471 - val_loss: 0.9764 - val_accuracy: 0.5462
Epoch 3/10
61/61 [==============================] - 39s 639ms/step - loss: 0.7832 - accuracy: 0.6372 - val_loss: 0.9057 - val_accuracy: 0.5626
Epoch 4/10
61/61 [==============================] - 36s 592ms/step - loss: 0.5391 - accuracy: 0.7569 - val_loss: 1.0539 - val_accuracy: 0.5421
Epoch 5/10
61/61 [==============================] - 41s 669ms/step - loss: 0.4271 - accuracy: 0.7928 - val_loss: 1.0978 - val_accuracy: 0.5544
Epoch 6/10
61/61 [==============================] - 40s 658ms/step - loss: 0.3516 - accuracy: 0.8475 - val_loss: 1.5173 - val_accuracy: 0.5154
Epoch 7/10
61/61 [==============================] - 36s 584ms/step - loss: 0.2729 - accuracy: 0.8937 - val_loss: 1.7871 - val_accuracy: 0.4682

# *****----------- NN -----------*****

# MultiHeadAttention Transformer + EfficientNet 

# D3-Late Fusion* V08

In [13]:
import numpy as np
import time
from tensorflow.keras.layers import (Input, Embedding, MultiHeadAttention, LayerNormalization,
                                     GlobalAveragePooling1D, Dense, GlobalAveragePooling2D, Concatenate)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
start_time = time.time()
#------------------------------------------------------------
# Text Transformer Encoder
#------------------------------------------------------------
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=128)(text_input)
attn = MultiHeadAttention(num_heads=4, key_dim=32)(t, t)
attn = LayerNormalization()(attn)
attn = GlobalAveragePooling1D()(attn)
text_feat = Dense(64, activation='relu')(attn)


text_prob = Dense(3, activation='softmax', name='text_softmax')(text_feat)

#------------------------------------------------------------
# Image Encoder using EfficientNetB0
#------------------------------------------------------------
image_input = Input(shape=(64, 64, 3), name='image_input')
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=image_input)
base_model.trainable = False
i = base_model.output
i = GlobalAveragePooling2D()(i)
image_feat = Dense(64, activation='relu')(i)


image_prob = Dense(3, activation='softmax', name='image_softmax')(image_feat)

#------------------------------------------------------------
# Model (Dual outputs)
#------------------------------------------------------------
model = Model(inputs=[text_input, image_input], outputs=[text_prob, image_prob])


model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'text_softmax': 'categorical_crossentropy', 'image_softmax': 'categorical_crossentropy'},
    loss_weights={'text_softmax': 1.0, 'image_softmax': 1.0},
    metrics={'text_softmax': ['accuracy'], 'image_softmax': ['accuracy']}
)

#------------------------------------------------------------
# Training
#------------------------------------------------------------
history = model.fit(
    [X_train_text, X_train_image],
    [y_train, y_train],
    validation_data=([X_val_text, X_val_image], [y_val, y_val]),
    epochs=10,
    batch_size=64
)

#------------------------------------------------------------
# Evaluation — Late Fusion at inference
#------------------------------------------------------------
text_probs, image_probs = model.predict([X_test_text, X_test_image])

# Plan A：Average
ensemble_probs = 0.5 * (text_probs + image_probs)

# Plan B：
# import numpy as np
# logp = np.log(text_probs + 1e-12) + np.log(image_probs + 1e-12)
# ensemble_probs = np.exp(logp)
# ensemble_probs /= ensemble_probs.sum(axis=1, keepdims=True)

predictions = np.argmax(ensemble_probs, axis=1)
true_labels = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report
print("\nClassification Report (Late Fusion):")
print(classification_report(true_labels, predictions, digits=4))


end_time = time.time()


elapsed_time = end_time - start_time


print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")


Epoch 1/10
61/61 [==============================] - 116s 2s/step - loss: 2.0085 - text_softmax_loss: 1.0086 - image_softmax_loss: 0.9998 - text_softmax_accuracy: 0.5345 - image_softmax_accuracy: 0.5435 - val_loss: 2.0097 - val_text_softmax_loss: 1.0066 - val_image_softmax_loss: 1.0031 - val_text_softmax_accuracy: 0.5462 - val_image_softmax_accuracy: 0.5462
Epoch 2/10
61/61 [==============================] - 120s 2s/step - loss: 1.9994 - text_softmax_loss: 0.9965 - image_softmax_loss: 1.0029 - text_softmax_accuracy: 0.5466 - image_softmax_accuracy: 0.5451 - val_loss: 2.0028 - val_text_softmax_loss: 1.0065 - val_image_softmax_loss: 0.9963 - val_text_softmax_accuracy: 0.5462 - val_image_softmax_accuracy: 0.5462
Epoch 3/10
61/61 [==============================] - 130s 2s/step - loss: 1.8873 - text_softmax_loss: 0.8962 - image_softmax_loss: 0.9911 - text_softmax_accuracy: 0.5931 - image_softmax_accuracy: 0.5492 - val_loss: 1.8891 - val_text_softmax_loss: 0.8858 - val_image_softmax_loss: 1.0

### 》》 D3 Logit

In [9]:
import numpy as np
import time
from tensorflow.keras.layers import (Input, Embedding, MultiHeadAttention, LayerNormalization,
                                     GlobalAveragePooling1D, Dense, GlobalAveragePooling2D, Concatenate)
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report
start_time = time.time()
#------------------------------------------------------------
# Text branch -> own classifier
#------------------------------------------------------------
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=128)(text_input)
t = MultiHeadAttention(num_heads=4, key_dim=32)(t, t)
t = LayerNormalization()(t)
t = GlobalAveragePooling1D()(t)
t_feat = Dense(64, activation='relu', name='text_feat')(t)
text_logits = Dense(3, activation=None, name='text_logits')(t_feat)     # logits
text_probs  = Dense(3, activation='softmax', name='text_softmax')(t_feat)  # probs 

#------------------------------------------------------------
# Image branch -> own classifier
#------------------------------------------------------------
image_input = Input(shape=(64, 64, 3), name='image_input')
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=image_input)
base_model.trainable = False
i = base_model.output
i = GlobalAveragePooling2D()(i)
i_feat = Dense(64, activation='relu', name='image_feat')(i)
image_logits = Dense(3, activation=None, name='image_logits')(i_feat)
image_probs  = Dense(3, activation='softmax', name='image_softmax')(i_feat)

#------------------------------------------------------------
# Build multi-output model 
#------------------------------------------------------------
model = Model(inputs=[text_input, image_input],
              outputs=[text_probs, image_probs])  # or [text_logits, image_logits]

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={'text_softmax':'categorical_crossentropy', 'image_softmax':'categorical_crossentropy'},
    loss_weights={'text_softmax':1.0, 'image_softmax':1.0},
    metrics={'text_softmax':['accuracy'], 'image_softmax':['accuracy']}
)

#------------------------------------------------------------
# Training 
#------------------------------------------------------------
history = model.fit(
    [X_train_text, X_train_image],
    {'text_softmax': y_train, 'image_softmax': y_train},
    validation_data=([X_val_text, X_val_image], {'text_softmax': y_val, 'image_softmax': y_val}),
    epochs=10,
    batch_size=64
)

#------------------------------------------------------------
# Prediction & LATE FUSION
#------------------------------------------------------------
text_probs, image_probs = model.predict([X_test_text, X_test_image], batch_size=64)

# 1) Ave
ensemble_probs_avg = 0.5 * text_probs + 0.5 * image_probs

# 2) log：softmax(log p1 + log p2)
logp_t = np.log(text_probs + 1e-12)
logp_i = np.log(image_probs + 1e-12)
ensemble_probs_geo = np.exp(logp_t + logp_i)
ensemble_probs_geo = ensemble_probs_geo / ensemble_probs_geo.sum(axis=1, keepdims=True)

# 3) With Weighting α（eg α∈[0,1]）
alpha = 0.6  
ensemble_probs_alpha = alpha * text_probs + (1 - alpha) * image_probs

pred = np.argmax(ensemble_probs_geo, axis=1)
true_labels = np.argmax(y_test, axis=1)
print(classification_report(true_labels, pred, digits=4))

end_time = time.time()

elapsed_time = end_time - start_time

print(f"EARLY FUSION TIME TAKEN：{elapsed_time:.4f} 秒")

Epoch 1/10
61/61 [==============================] - 110s 2s/step - loss: 2.0042 - text_softmax_loss: 1.0025 - image_softmax_loss: 1.0017 - text_softmax_accuracy: 0.5409 - image_softmax_accuracy: 0.5448 - val_loss: 1.9962 - val_text_softmax_loss: 0.9986 - val_image_softmax_loss: 0.9975 - val_text_softmax_accuracy: 0.5462 - val_image_softmax_accuracy: 0.5462
Epoch 2/10
61/61 [==============================] - 117s 2s/step - loss: 1.9729 - text_softmax_loss: 0.9792 - image_softmax_loss: 0.9937 - text_softmax_accuracy: 0.5492 - image_softmax_accuracy: 0.5492 - val_loss: 1.9832 - val_text_softmax_loss: 0.9812 - val_image_softmax_loss: 1.0020 - val_text_softmax_accuracy: 0.5483 - val_image_softmax_accuracy: 0.5462
Epoch 3/10
61/61 [==============================] - 112s 2s/step - loss: 1.7907 - text_softmax_loss: 0.7971 - image_softmax_loss: 0.9936 - text_softmax_accuracy: 0.6580 - image_softmax_accuracy: 0.5492 - val_loss: 1.9197 - val_text_softmax_loss: 0.9169 - val_image_softmax_loss: 1.0

> # Late Fusion

## Transformer+ EfficentNet 

> ## D4 Early Fusion

In [35]:
from keras.layers import LSTM
from keras import regularizers
from keras.optimizers import Adam, Adagrad, RMSprop
from keras import backend as K
from keras import metrics
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, BatchNormalization, Flatten, Dense
from tensorflow.keras.regularizers import l2
import time
from sklearn.metrics import classification_report
import numpy as np
from tensorflow.keras.layers import Input, Embedding, MultiHeadAttention, GlobalAveragePooling1D, LayerNormalization, Concatenate
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Model

#------------------------------------------------------------
# Timer Start
#------------------------------------------------------------
start_time = time.time()

def some_function():
    result = 0
    for i in range(1000000):
        result += i
    return result

#------------------------------------------------------------
# Text Transformer Encoder
#------------------------------------------------------------
text_input = Input(shape=(200,), name='text_input')
t = Embedding(input_dim=10000, output_dim=128)(text_input)
t = MultiHeadAttention(num_heads=4, key_dim=32)(t, t)
t = LayerNormalization()(t)
t = GlobalAveragePooling1D()(t)

#------------------------------------------------------------
# Image Encoder using EfficientNetB0
#------------------------------------------------------------
image_input = Input(shape=(64, 64, 3), name='image_input')
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=image_input)
base_model.trainable = False
i = base_model.output
i = GlobalAveragePooling2D()(i)

#------------------------------------------------------------
# Early Fusion
#------------------------------------------------------------
concatenated = Concatenate()([t, i])
fusion = Dense(64, activation='relu')(concatenated)
fusion = Dense(64, activation='relu')(fusion)
output = Dense(3, activation='softmax')(fusion)

model = Model(inputs=[text_input, image_input], outputs=output)
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

#------------------------------------------------------------
# Training
#------------------------------------------------------------
history = model.fit([X_train_text, X_train_image], y_train, 
                    validation_data=([X_val_text, X_val_image], y_val),
                    epochs=10, 
                    batch_size=64)

#------------------------------------------------------------
# Evaluation
#------------------------------------------------------------
predictions_prob = model.predict([X_test_text, X_test_image])
predictions = np.argmax(predictions_prob, axis=1)
true_labels = np.argmax(y_test, axis=1)

print("\nClassification Report:")
print(classification_report(true_labels, predictions, digits=4))
#------------------------------------------------------------
# Timer End
#------------------------------------------------------------
output = some_function()
print(f"\n函數輸出: {output}")
end_time = time.time()
elapsed_time = end_time - start_time
print(f"程式碼執行時間：{elapsed_time:.4f} 秒")


Epoch 1/10
61/61 [==============================] - 101s 2s/step - loss: 0.9965 - accuracy: 0.5492 - val_loss: 0.9965 - val_accuracy: 0.5462
Epoch 2/10
61/61 [==============================] - 116s 2s/step - loss: 0.9904 - accuracy: 0.5492 - val_loss: 1.0003 - val_accuracy: 0.5462
Epoch 3/10
61/61 [==============================] - 113s 2s/step - loss: 0.9889 - accuracy: 0.5492 - val_loss: 0.9976 - val_accuracy: 0.5462
Epoch 4/10
61/61 [==============================] - 116s 2s/step - loss: 0.9372 - accuracy: 0.5546 - val_loss: 0.8993 - val_accuracy: 0.5749
Epoch 5/10
61/61 [==============================] - 118s 2s/step - loss: 0.7254 - accuracy: 0.6770 - val_loss: 0.9828 - val_accuracy: 0.5852
Epoch 6/10
61/61 [==============================] - 115s 2s/step - loss: 0.5560 - accuracy: 0.7484 - val_loss: 1.1301 - val_accuracy: 0.5708
Epoch 7/10
61/61 [==============================] - 120s 2s/step - loss: 0.4727 - accuracy: 0.7792 - val_loss: 1.2314 - val_accuracy: 0.5729
Epoch 8/10
61